In [15]:
%load_ext autoreload
%autoreload 2
import os
import torch
import json
from glob import glob
from PIL import Image
from pytorch3d.io import IO
import numpy as np
from src.utils import normalize_pc
from src.render_pc import render_pc_no_vis
from src.glip_inference import glip_inference_noLoad, load_model
from src.gen_superpoint import gen_superpoint
from src.bbox2seg import bbox2seg
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
config ="GLIP/configs/glip_Swin_L.yaml"
weight_path = "models/glip_large_model.pth"
glip_demo = load_model(config, weight_path)


/rhome/kamburoglu/miniconda3/envs/partslip/lib/python3.9/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'bert.pooler.dense.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NO

KeyboardInterrupt: 

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)

device

device(type='cuda', index=0)

In [8]:
def load_imgs(file_name):
        pil_images = []

        for i in range(6):
            pil_images.append(np.array(Image.open(f"{file_name}/images/{i}.png").convert("RGB"))[:, :, [2, 1, 0]])
        return pil_images

class ShapeNetParts(torch.utils.data.Dataset):
    metaData = json.load(open("./PartNetE_meta.json",'r'))
    def __init__(self, split):
        assert split in ['train', 'val', 'test']
        self.items = glob(f"./data/*/*")
    def __getitem__(self, index):
        path = self.items[index]

        #pointcloudImages = load_imgs(path)
        segmentation_labels = np.load(f"{path}/label.npy",allow_pickle=True).item()
        return {
            'class': path.split("/")[-2],
            'path': path,
            'segmentation_labels': segmentation_labels["semantic_seg"]
        }
    def __len__(self):
        return len(self.items)

In [9]:
dataset_test = ShapeNetParts("test")
dataloader_test = torch.utils.data.dataloader.DataLoader(dataset_test,batch_size=1,shuffle=False)

In [10]:
io = IO()

In [15]:
correct_p = 0
total_p = 0
for batch in tqdm(dataloader_test):
    xyz, rgb = normalize_pc(pc_file=batch["path"][0], save_dir="",io=io, device=device)
    img_list, pc_idx, screen_coords = render_pc_no_vis(xyz, rgb, device)
    preds = glip_inference_noLoad(glip_demo,img_list ,ShapeNetParts.metaData[batch["class"][0]])
    superpoint = gen_superpoint(xyz, rgb, visualize=False, save_dir="")
    sem_seg, ins_seg = bbox2seg(xyz, superpoint, preds, screen_coords, pc_idx, ShapeNetParts.metaData[batch["class"][0]], "test",solve_instance_seg=False,visualize=False)
    correct_p += (sem_seg == batch["segmentation_labels"][0].numpy()).sum()
    total_p += sem_seg.shape[0]

  0%|                                                                                          | 0/1906 [00:00<?, ?it/s]/rhome/kamburoglu/miniconda3/envs/partslip/lib/python3.9/site-packages/pytorch3d/transforms/transform3d.py:800: UserWarning: R is not a valid rotation matrix
  warnings.warn(msg)
/rhome/kamburoglu/miniconda3/envs/partslip/lib/python3.9/site-packages/transformers/modeling_utils.py:884: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/rhome/kamburoglu/miniconda3/envs/partslip/lib/python3.9/site-packages/torch/nn/functional.py:3825: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.")
  0%|                                                                               | 1/1906 [01:18<41:20:40, 78.13s/it]/rhome/kamburoglu/miniconda3/envs/partslip/lib/python3.9/si

KeyboardInterrupt: 

In [ ]:
print("Correctly Labeled #Points: ",correct_p)
print("Total #Points: ",total_p)
print("Accuracy: %",100*correct_p/total_p)